# ME simulations of the transcription cost

## Import

In [1]:
import random
from cobrame.util import building, mu, me_model_interface

import pickle
import pandas as pd
import statistics 


## Modified functions from ECOLIme

In [2]:
# Function to solve the model at an specific mu (0.69)

def solve_me_model_fixed(me, muf,using_soplex=False):
    if using_soplex:
        from cobrame.solve.algorithms import solve_at_growth_rate
        solve_at_growth_rate(me, muf)
    else:
        from qminospy.me1 import ME_NLP1
        print('qminos')
        # The object containing solveME methods--composite that uses a ME model object 
        me_nlp = ME_NLP1(me, growth_key='mu')
        # Use solv for now
        xopt,status,hs = me_nlp.solvelp(0.69, verbosity=2)
        me.solution.f = me.solution.x_dict['biomass_dilution']

## Simulations

In [3]:
# Function to add a number of extra genes that are transcribed but not translated

def create_genes(num_genes,valor_bound,transcription = True):
    with open('../../files/models/iJL1678b.pickle', 'rb') as f:
        me = pickle.load(f)
    
    lista_genes={}
    seq = "ATG"
    codons = pd.read_csv("../../files/ecolime_data/codon_usage.csv", index_col=0)
    for codon, row in codons.iterrows():
        if row.amino_acid == "Stop":
            continue
        seq += codon * int(row.per_1000 // 3)  #want roughly 300 aa
    # get the most used stop codon
    seq += codons[codons.amino_acid == "Stop"].sort_values("per_1000").index[-1]

    for n in range(1, num_genes+1):
        lista_genes['random_'+str(n)] = seq    
    
    # Add details of the transcription process
    for gen,secuencia in lista_genes.items():
        building.create_transcribed_gene(me, gen, 'mRNA', secuencia)
        if transcription:
            building.add_transcription_reaction(me, 'TU_'+gen,{gen}, secuencia)
            me.process_data.get_by_id('TU_'+gen).RNA_polymerase = "RNAP70-CPLX"
            me.reactions.get_by_id('transcription_TU_'+gen).lower_bound =valor_bound
            me.reactions.get_by_id('transcription_TU_'+gen).update()
            
    not_prune = []
    
    if transcription:    
        not_prune = ['transcription_TU_'+n for n in lista_genes.keys()]
        
    not_prune.extend(['RNA_'+n for n in lista_genes.keys()])
        
    me.update()
    me.prune(not_prune)
    
    # Constrains of the glucose and oxygen consumption
    me.reactions.get_by_id("EX_glc__D_e").upper_bound = -10.
    me.reactions.get_by_id("EX_glc__D_e").lower_bound = -10.
    me.reactions.get_by_id("EX_o2_e").lower_bound = -18.
    me.reactions.get_by_id("EX_o2_e").upper_bound = -18.
    
    solve_me_model_fixed(me, 0.69)
    
    # Save the model
    with open('../../files/models/AumGenes_GLC_OX_'+str(num_genes)+'.pickle', 'wb') as f:
        pickle.dump(me, f)
        

In [6]:
with open('../../files/models/iJL1678b.pickle', 'rb') as f:
    me = pickle.load(f)
    
me.update()
me.prune()

solve_me_model_fixed(me, 0.69)

/home/notebooks/cobrame/cobrame/core/processdata.py:256 UserWarning: met (generic_tRNA_GAG_glu__L_c) does not have formula


qminos
Finished compiling expressions in 169.354191 seconds
Finished substituting S,lb,ub in 4.054594 seconds
Finished makeME_LP in 0.760675 seconds
Getting MINOS parameters from ME_NLP...


In [7]:
# Number of genes to be added (Approximately a 5%,10%, 15% y 20% of the 4600 genes)
#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4135669/

num_genes =[230,460,690,920]

valores_trans = list(me.get_transcription_flux().values())
exp_mean = statistics.mean([n for n in valores_trans if n != 0]) # promedio de expresión 

for n in num_genes:
    create_genes(n, exp_mean)

/home/notebooks/cobrame/cobrame/core/reaction.py:852 UserWarning: RNA Polymerase () not found
/home/notebooks/cobrame/cobrame/core/processdata.py:256 UserWarning: met (generic_tRNA_GAG_glu__L_c) does not have formula


qminos
Finished compiling expressions in 167.191147 seconds
Finished substituting S,lb,ub in 3.809132 seconds
Finished makeME_LP in 0.662478 seconds
Getting MINOS parameters from ME_NLP...


/home/notebooks/cobrame/cobrame/core/reaction.py:852 UserWarning: RNA Polymerase () not found
/home/notebooks/cobrame/cobrame/core/processdata.py:256 UserWarning: met (generic_tRNA_GAG_glu__L_c) does not have formula


qminos
Finished compiling expressions in 169.675024 seconds
Finished substituting S,lb,ub in 3.969353 seconds
Finished makeME_LP in 0.668216 seconds
Getting MINOS parameters from ME_NLP...


/home/notebooks/cobrame/cobrame/core/reaction.py:852 UserWarning: RNA Polymerase () not found
/home/notebooks/cobrame/cobrame/core/processdata.py:256 UserWarning: met (generic_tRNA_GAG_glu__L_c) does not have formula


qminos
Finished compiling expressions in 174.647178 seconds
Finished substituting S,lb,ub in 4.583837 seconds
Finished makeME_LP in 0.886642 seconds
Getting MINOS parameters from ME_NLP...


/home/notebooks/cobrame/cobrame/core/reaction.py:852 UserWarning: RNA Polymerase () not found
/home/notebooks/cobrame/cobrame/core/processdata.py:256 UserWarning: met (generic_tRNA_GAG_glu__L_c) does not have formula


qminos
Finished compiling expressions in 173.152207 seconds
Finished substituting S,lb,ub in 4.619005 seconds
Finished makeME_LP in 0.816061 seconds
Getting MINOS parameters from ME_NLP...
